In [1]:
import os
os.chdir('..')
!pwd

/home/jupyter-kriach/md_github_lat/multigroupcode/multigroup-code


In [2]:
import pandas as pd
import numpy as np

In [3]:
from bilevel.utils import *

# Exploratory analyis of reconstructed Adult dataset

In [4]:
df_adult = pd.read_csv('./data_small/adult_reconstruction.csv')

In [5]:
df_adult.shape

(49531, 14)

In [6]:
df_adult.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,20,40,0,0,Private,Bachelors,13,Married-civ-spouse,Wife,White,Female,United-States,49100,Tech-support
1,40,21,0,0,Private,Some-college,10,Divorced,Own-child,White,Male,United-States,11500,Craft-repair
2,10,17,0,0,Private,11th,7,Never-married,Own-child,White,Male,United-States,2600,Other-service
3,50,51,0,0,Private,HS-grad,9,Married-civ-spouse,Husband,Asian-Pac-Islander,Male,Cambodia,38997,Sales
4,38,28,0,0,Private,Bachelors,13,Never-married,Not-in-family,White,Male,?,41400,Exec-managerial


In [7]:
df_adult.isnull().any(axis=1)

0        False
1        False
2        False
3        False
4        False
         ...  
49526    False
49527    False
49528    False
49529    False
49530    False
Length: 49531, dtype: bool

In [8]:
df_adult.isnull().sum()

hours-per-week    0
age               0
capital-gain      0
capital-loss      0
workclass         0
education         0
education-num     0
marital-status    0
relationship      0
race              0
sex               0
native-country    0
income            0
occupation        0
dtype: int64

In [9]:
df_adult.columns

Index(['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'workclass',
       'education', 'education-num', 'marital-status', 'relationship', 'race',
       'sex', 'native-country', 'income', 'occupation'],
      dtype='object')

In [10]:
numeric_feat = ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num']
cat_feat = ['workclass', 'education', 'marital-status', 'relationship', 'race', 'sex', 'native-country', 'occupation']
print(len(numeric_feat), len(cat_feat))

5 8


In [11]:
# categoricals
from collections import Counter
for c in cat_feat:
    dic = Counter(df_adult[c])
    str = ""
    if '?' in dic:
        str = '?'
    print(c, str ,Counter(df_adult[c]))

# Takewaya workclass, native country, occupation have '?' category for unknown


workclass ? Counter({'Private': 34486, 'Self-emp-not-inc': 3871, 'Local-gov': 3157, '?': 2859, 'State-gov': 1990, 'Self-emp-inc': 1698, 'Federal-gov': 1439, 'Without-pay': 21, 'Never-worked': 10})
education  Counter({'HS-grad': 16086, 'Some-college': 11066, 'Bachelors': 8109, 'Masters': 2668, 'Assoc-voc': 2066, '11th': 1852, 'Assoc-acdm': 1608, '10th': 1405, '7th-8th': 962, 'Prof-school': 835, '9th': 761, '12th': 662, 'Doctorate': 595, '5th-6th': 525, '1st-4th': 248, 'Preschool': 83})
marital-status  Counter({'Married-civ-spouse': 22737, 'Never-married': 16398, 'Divorced': 6679, 'Separated': 1532, 'Widowed': 1519, 'Married-spouse-absent': 629, 'Married-AF-spouse': 37})
relationship  Counter({'Husband': 20065, 'Not-in-family': 12699, 'Own-child': 7764, 'Unmarried': 5151, 'Wife': 2339, 'Other-relative': 1513})
race  Counter({'White': 42441, 'Black': 4694, 'Asian-Pac-Islander': 1519, 'Amer-Indian-Eskimo': 471, 'Other': 406})
sex  Counter({'Male': 33174, 'Female': 16357})
native-country ? 

In [12]:
# numerics
df_adult[numeric_feat].describe()

,hours-per-week,age,capital-gain,capital-loss,education-num
count,49531.000000,49531.000000,49531.000000,49531.000000,49531.000000
mean,40.399184,38.554320,1064.819144,86.285114,10.069795
std,12.340824,13.701025,7401.671807,400.322887,2.565829
min,1.000000,17.000000,0.000000,0.000000,1.000000
25%,40.000000,28.000000,0.000000,0.000000,9.000000
50%,40.000000,37.000000,0.000000,0.000000,10.000000
75%,45.000000,48.000000,0.000000,0.000000,12.000000
max,99.000000,90.000000,99999.000000,4356.000000,16.000000


In [13]:
df_adult['income'].describe()

count    49531.000000
mean     33509.843129
std      27051.436074
min        107.000000
25%      12027.000000
50%      26013.000000
75%      48457.500000
max      99999.000000
Name: income, dtype: float64

In [14]:
# groups
race_groups = df_adult['race'].unique()
sex_groups = df_adult['sex'].unique()
race_groups, sex_groups

(array(['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other',
        'Black'], dtype=object),
 array(['Female', 'Male'], dtype=object))

In [15]:
numerics_all = numeric_feat + ['income']
print(numerics_all)

['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num', 'income']


In [16]:
df_adult_mm = numeric_scaler(df_adult, numerics_all)
df_adult_mm.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,0.193878,0.315068,0.0,0.0,Private,Bachelors,0.800000,Married-civ-spouse,Wife,White,Female,United-States,0.490460,Tech-support
1,0.397959,0.054795,0.0,0.0,Private,Some-college,0.600000,Divorced,Own-child,White,Male,United-States,0.114053,Craft-repair
2,0.091837,0.000000,0.0,0.0,Private,11th,0.400000,Never-married,Own-child,White,Male,United-States,0.024957,Other-service
3,0.500000,0.465753,0.0,0.0,Private,HS-grad,0.533333,Married-civ-spouse,Husband,Asian-Pac-Islander,Male,Cambodia,0.389320,Sales
4,0.377551,0.150685,0.0,0.0,Private,Bachelors,0.800000,Never-married,Not-in-family,White,Male,?,0.413376,Exec-managerial


In [17]:
df_adult_mm_le = ordinal_encoder(df_adult_mm, cat_feat)
df_adult_mm_le.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,0.193878,0.315068,0.0,0.0,4.0,9.0,0.800000,2.0,5.0,4.0,0.0,39.0,0.490460,13.0
1,0.397959,0.054795,0.0,0.0,4.0,15.0,0.600000,0.0,3.0,4.0,1.0,39.0,0.114053,3.0
2,0.091837,0.000000,0.0,0.0,4.0,1.0,0.400000,4.0,3.0,4.0,1.0,39.0,0.024957,8.0
3,0.500000,0.465753,0.0,0.0,4.0,11.0,0.533333,2.0,0.0,1.0,1.0,1.0,0.389320,12.0
4,0.377551,0.150685,0.0,0.0,4.0,9.0,0.800000,4.0,1.0,4.0,1.0,0.0,0.413376,4.0


In [18]:
df_adult_mm_oh = one_hot(df_adult_mm, cat_feat)
df_adult_mm_oh.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh = df_adult_mm_oh * 1.0 # make bool true, false into 1.0, 0.0
df_adult_mm_oh.head()

,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,0.193878,0.315068,0.0,0.0,0.800000,0.490460,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.397959,0.054795,0.0,0.0,0.600000,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.091837,0.000000,0.0,0.0,0.400000,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.500000,0.465753,0.0,0.0,0.533333,0.389320,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.377551,0.150685,0.0,0.0,0.800000,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Benchmarking ML batch algorithms

In [19]:
from sklearn.model_selection import train_test_split
random_seed = 21
X_train, X_test, y_train, y_test = train_test_split(df_adult_mm_oh.drop('income', axis=1), 
                                                    df_adult_mm_oh['income'], test_size = 0.2,
                                                    shuffle=True, random_state= random_seed)

In [20]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree

In [21]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print(lr.score(X_train, y_train), lr.score(X_test, y_test))

0.5839699444945836 -214714363240219.47


In [22]:
ridge = Ridge()
ridge.fit(X_train, y_train)
print(ridge.score(X_train, y_train), ridge.score(X_test, y_test))

0.5839681387328852 0.5835313399697666


In [24]:
df_le = pd.read_pickle('./data_small/dataframes/df_adultog_le.pkl')
random_seed = 21
X_train_le, X_test_le, y_train, y_test = train_test_split(df_le.drop('income', axis=1), 
                                                    df_le['income'], test_size = 0.2,
                                                    shuffle=True, random_state= random_seed)

In [25]:
X_train_le.shape, y_train.shape, X_test_le.shape, y_test.shape

((39624, 13), (39624,), (9907, 13), (9907,))

In [26]:
# gbr = GradientBoostingRegressor(max_depth = 10, n_estimators = 1000, learning_rate = 0.05)
gbr = GradientBoostingRegressor(max_depth = 4, n_estimators = 700, learning_rate = 0.1)
gbr.fit(X_train_le, y_train)
print(gbr.score(X_train_le, y_train), gbr.score(X_test_le, y_test))

0.7320034074105475 0.6985286344011372


In [ ]:
# # grid search for ``best'' gbr, in hyperopt/Adult_gridgbr.py
# param_grid = {
# "max_depth" = np.arange(3, 20),
# "learning_rate" = [0.01, 0.05, 0.1],
# "n_estimators" = np.arange(100, 1100, 200)
# }
# gbr_search = GradientBoostingRegressor()
# grid = GridSearchCV(estimator = gbr_search, 
#                     param_grid = param_grid, 
#                     scoring='r2',
#                     cv=5,
#                     n_jobs = 4, 
#                     verbose = 10)
# grid_result = grid_search.fit(X_train, y_train)


In [27]:
np.arange(100, 1100, 200)

array([100, 300, 500, 700, 900])

In [34]:
dtree_reg = tree.DecisionTreeRegressor(max_depth=11)
dtree_reg.fit(X_train_le, y_train)
print(dtree_reg.score(X_train_le, y_train), dtree_reg.score(X_test_le, y_test))

0.6887384232609746 0.6347359260510592


In [56]:
from sklearn.neighbors import KNeighborsRegressor
knn10 = KNeighborsRegressor(n_neighbors = 20) # not even as good as ridge
knn10.fit(X_train, y_train)
print(knn10.score(X_train, y_train), knn10.score(X_test, y_test))

0.6157087711671563 0.5791236932385222


# Data preparation sequence for ACS reconstructed

1. Sort by RACE, SEX or not
2. minmax scale numerics ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num', 'income']

----
categorics ['workclass', 'education', 'marital-status', 'relationship', 'race', 'sex', 'native-country', 'occupation']

3. one hot encode, drop categories, save A_t, drop sensitive or not, shuffle seed
3. label encode, use the same above A_t, drop sensitive or not, use same above shuffle

## unshuffled original ordering in adult reconstructed, one hot and label encoded
- not sorted by sensitive groups(race, sex)
- just minmax scaled, numeric
- one hot encode, drop cat og

In [5]:
numeric_all = ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num', 'income']
cat_feat = ['workclass', 'education', 'marital-status', 'relationship', 'race', 'sex', 'native-country', 'occupation']
print(len(numeric_all), len(cat_feat))

6 8


In [6]:
df_adult = pd.read_csv('./data_small/adult_reconstruction.csv')

In [10]:
df_adult_mm = numeric_scaler(df_adult, numeric_all)
df_adult_mm_oh = one_hot(df_adult_mm, cat_feat)
df_adult_mm_oh.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh = df_adult_mm_oh * 1.0 # make bool true, false into 1.0, 0.0
df_adult_mm_oh.head()

,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,0.193878,0.315068,0.0,0.0,0.800000,0.490460,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.397959,0.054795,0.0,0.0,0.600000,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.091837,0.000000,0.0,0.0,0.400000,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.500000,0.465753,0.0,0.0,0.533333,0.389320,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.377551,0.150685,0.0,0.0,0.800000,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
race_groups = []
sex_groups = []
for col in df_adult_mm_oh.columns:
    if 'race' in col:
        race_groups.append(col)
    if 'sex' in col:
        sex_groups.append(col)
print(race_groups, sex_groups)

(['race_Amer-Indian-Eskimo',
  'race_Asian-Pac-Islander',
  'race_Black',
  'race_Other',
  'race_White'],
 ['sex_Female', 'sex_Male'])

## the sensitive groups

In [16]:
race_groups+sex_groups

['race_Amer-Indian-Eskimo',
 'race_Asian-Pac-Islander',
 'race_Black',
 'race_Other',
 'race_White',
 'sex_Female',
 'sex_Male']

In [15]:
def get_A_t_from_onehotdf(df_oh, groups, always_on = False) -> np.ndarray:
    A_tdf = df_oh[groups]
    if always_on:
        A_tdf['alwayson'] = 1
    return A_tdf.to_numpy()

In [18]:
A_tog = get_A_t_from_onehotdf(df_adult_mm_oh, race_groups+sex_groups, always_on = True)

/tmp/ipykernel_3191807/1068999074.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A_tdf['alwayson'] = 1


In [21]:
np.save('./data_small/nparrays/A_t_adultog', A_tog)

In [24]:
df_adult_mm_oh.to_pickle('./data_small/dataframes/df_adultog_oh.pkl') # mm then oh (dropped categorics)

In [35]:
df_adult_mm_le = ordinal_encoder(df_adult_mm, cat_feat)
df_adult_mm_le.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,0.193878,0.315068,0.0,0.0,4.0,9.0,0.800000,2.0,5.0,4.0,0.0,39.0,0.490460,13.0
1,0.397959,0.054795,0.0,0.0,4.0,15.0,0.600000,0.0,3.0,4.0,1.0,39.0,0.114053,3.0
2,0.091837,0.000000,0.0,0.0,4.0,1.0,0.400000,4.0,3.0,4.0,1.0,39.0,0.024957,8.0
3,0.500000,0.465753,0.0,0.0,4.0,11.0,0.533333,2.0,0.0,1.0,1.0,1.0,0.389320,12.0
4,0.377551,0.150685,0.0,0.0,4.0,9.0,0.800000,4.0,1.0,4.0,1.0,0.0,0.413376,4.0


In [36]:
# np.all((A_tog[:,-3] == df_adult_mm_le['sex']) == False) 

True

In [37]:
df_adult_mm_le.to_pickle('./data_small/dataframes/df_adultog_le.pkl') # mm then label encoded

## sort by race, largest to smallest; mergesort

In [38]:
from collections import Counter
Counter(df_adult['sex']), Counter(df_adult['race'])

(Counter({'Male': 33174, 'Female': 16357}),
 Counter({'White': 42441,
          'Black': 4694,
          'Asian-Pac-Islander': 1519,
          'Amer-Indian-Eskimo': 471,
          'Other': 406}))

In [39]:
Counter(df_adult_mm['sex']), Counter(df_adult_mm['race'])

(Counter({'Male': 33174, 'Female': 16357}),
 Counter({'White': 42441,
          'Black': 4694,
          'Asian-Pac-Islander': 1519,
          'Amer-Indian-Eskimo': 471,
          'Other': 406}))

In [42]:
df_adult_mm_racels = df_adult_mm.sort_values(by='race', key=lambda x: x.map(x.value_counts()), ascending=False, inplace = False, kind = 'mergesort')

In [43]:
df_adult_mm_racels

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,0.193878,0.315068,0.0,0.00000,Private,Bachelors,0.800000,Married-civ-spouse,Wife,White,Female,United-States,0.490460,Tech-support
1,0.397959,0.054795,0.0,0.00000,Private,Some-college,0.600000,Divorced,Own-child,White,Male,United-States,0.114053,Craft-repair
2,0.091837,0.000000,0.0,0.00000,Private,11th,0.400000,Never-married,Own-child,White,Male,United-States,0.024957,Other-service
4,0.377551,0.150685,0.0,0.00000,Private,Bachelors,0.800000,Never-married,Not-in-family,White,Male,?,0.413376,Exec-managerial
5,0.377551,0.123288,0.0,0.43067,Private,Bachelors,0.800000,Never-married,Not-in-family,White,Male,United-States,0.384585,Exec-managerial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48741,0.295918,0.164384,0.0,0.00000,Private,HS-grad,0.533333,Divorced,Not-in-family,Other,Female,Germany,0.241190,Prof-specialty
48927,0.397959,0.246575,0.0,0.00000,Local-gov,Some-college,0.600000,Married-civ-spouse,Husband,Other,Male,United-States,0.822318,Protective-serv
49115,0.704082,0.205479,0.0,0.00000,Private,Some-college,0.600000,Separated,Unmarried,Other,Female,United-States,0.200176,Other-service
49117,0.397959,0.109589,0.0,0.00000,Private,HS-grad,0.533333,Married-civ-spouse,Own-child,Other,Male,United-States,0.299253,Transport-moving


In [44]:
df_adult_mm_oh_racels = one_hot(df_adult_mm_racels, cat_feat)
df_adult_mm_oh_racels.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh_racels = df_adult_mm_oh_racels * 1.0 # make bool true, false into 1.0, 0.0
df_adult_mm_oh_racels.head()

,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,0.193878,0.315068,0.0,0.00000,0.8,0.490460,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.397959,0.054795,0.0,0.00000,0.6,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.091837,0.000000,0.0,0.00000,0.4,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.377551,0.150685,0.0,0.00000,0.8,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.377551,0.123288,0.0,0.43067,0.8,0.384585,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
df_adult_mm_oh_racels.to_pickle('./data_small/dataframes/df_adult_racesorted_oh.pkl') # mm, sorted by race large-small, one hot 

In [46]:
race_groups+sex_groups

['race_Amer-Indian-Eskimo',
 'race_Asian-Pac-Islander',
 'race_Black',
 'race_Other',
 'race_White',
 'sex_Female',
 'sex_Male']

In [47]:
A_t_racels = get_A_t_from_onehotdf(df_adult_mm_oh_racels, race_groups + sex_groups, always_on = True)

/tmp/ipykernel_3191807/1068999074.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A_tdf['alwayson'] = 1


In [48]:
np.save('./data_small/nparrays/A_t_racels', A_t_racels)

In [50]:
# np.all(df_adult_mm_oh_racels['race_White'] == A_t_racels[:, 4])

True

In [53]:
df_adult_mm_le_racels = ordinal_encoder(df_adult_mm_racels, cat_feat)
df_adult_mm_le_racels.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,0.193878,0.315068,0.0,0.00000,4.0,9.0,0.8,2.0,5.0,4.0,0.0,39.0,0.490460,13.0
1,0.397959,0.054795,0.0,0.00000,4.0,15.0,0.6,0.0,3.0,4.0,1.0,39.0,0.114053,3.0
2,0.091837,0.000000,0.0,0.00000,4.0,1.0,0.4,4.0,3.0,4.0,1.0,39.0,0.024957,8.0
4,0.377551,0.150685,0.0,0.00000,4.0,9.0,0.8,4.0,1.0,4.0,1.0,0.0,0.413376,4.0
5,0.377551,0.123288,0.0,0.43067,4.0,9.0,0.8,4.0,1.0,4.0,1.0,39.0,0.384585,4.0


In [54]:
df_adult_mm_le_racels.to_pickle('./data_small/dataframes/df_adult_racesorted_le.pkl')

## sorted by sex, largest to smallest

In [56]:
df_adult_mm_sexls = df_adult_mm.sort_values(by='sex', key=lambda x: x.map(x.value_counts()), ascending=False, 
                                             inplace = False, kind = 'mergesort')
df_adult_mm_sexls.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
1,0.397959,0.054795,0.0,0.00000,Private,Some-college,0.600000,Divorced,Own-child,White,Male,United-States,0.114053,Craft-repair
2,0.091837,0.000000,0.0,0.00000,Private,11th,0.400000,Never-married,Own-child,White,Male,United-States,0.024957,Other-service
3,0.500000,0.465753,0.0,0.00000,Private,HS-grad,0.533333,Married-civ-spouse,Husband,Asian-Pac-Islander,Male,Cambodia,0.389320,Sales
4,0.377551,0.150685,0.0,0.00000,Private,Bachelors,0.800000,Never-married,Not-in-family,White,Male,?,0.413376,Exec-managerial
5,0.377551,0.123288,0.0,0.43067,Private,Bachelors,0.800000,Never-married,Not-in-family,White,Male,United-States,0.384585,Exec-managerial


In [57]:
df_adult_mm_oh_sexls = one_hot(df_adult_mm_sexls, cat_feat)
df_adult_mm_oh_sexls.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh_sexls = df_adult_mm_oh_sexls * 1.0 # make bool true, false into 1.0, 0.0
df_adult_mm_oh_sexls.head()

,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
1,0.397959,0.054795,0.0,0.00000,0.600000,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.091837,0.000000,0.0,0.00000,0.400000,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.500000,0.465753,0.0,0.00000,0.533333,0.389320,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.377551,0.150685,0.0,0.00000,0.800000,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.377551,0.123288,0.0,0.43067,0.800000,0.384585,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
df_adult_mm_oh_sexls.to_pickle('./data_small/dataframes/df_adult_sexsorted_oh.pkl')

In [60]:
sens_groups = race_groups + sex_groups

In [61]:
A_t_sexls = get_A_t_from_onehotdf(df_adult_mm_oh_sexls, sens_groups , always_on = True)

/tmp/ipykernel_3191807/1068999074.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A_tdf['alwayson'] = 1


In [64]:
# A_t_sexls[:, -2] == df_adult_mm_oh_sexls['sex_Male']

1        True
2        True
3        True
4        True
5        True
         ... 
49516    True
49517    True
49518    True
49529    True
49530    True
Name: sex_Male, Length: 49531, dtype: bool

In [65]:
np.save('./data_small/nparrays/A_t_sexls', A_t_sexls)

In [66]:
A_t_sexls.shape

(49531, 8)

In [67]:
df_adult_mm_le_sexls = ordinal_encoder(df_adult_mm_sexls, cat_feat)
df_adult_mm_le_sexls.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
1,0.397959,0.054795,0.0,0.00000,4.0,15.0,0.600000,0.0,3.0,4.0,1.0,39.0,0.114053,3.0
2,0.091837,0.000000,0.0,0.00000,4.0,1.0,0.400000,4.0,3.0,4.0,1.0,39.0,0.024957,8.0
3,0.500000,0.465753,0.0,0.00000,4.0,11.0,0.533333,2.0,0.0,1.0,1.0,1.0,0.389320,12.0
4,0.377551,0.150685,0.0,0.00000,4.0,9.0,0.800000,4.0,1.0,4.0,1.0,0.0,0.413376,4.0
5,0.377551,0.123288,0.0,0.43067,4.0,9.0,0.800000,4.0,1.0,4.0,1.0,39.0,0.384585,4.0


In [68]:
df_adult_mm_le_sexls.to_pickle('./data_small/dataframes/df_adult_sexsorted_le.pkl')